# 

# Data Science Capstone Project

<p>In this project we will analyze data about car accidents and will try to predict the severity of a new collision, if it's fatal or not</p>

### Importing libraries

In [16]:
# Pandas to handle dataset
import pandas as pd 
# Numpy to handle and operate through arrays
import numpy as np 

In [17]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
